In [1]:
from sqlalchemy import create_engine
from datetime import timedelta, date
from datetime import datetime
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import pymysql
import pickle
import csv
import os
import time

In [2]:
# tf gpu 메모리 관련 코드 
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
today = str(date.today() - timedelta(days=1)).replace('-','')
saveday = str(date.today() - timedelta(days=2)).replace('-','')
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn

#DB 내 존재하는 테이블(종목) 리스트 추출
def get_pymysql_stock_list(conn, db_name):

    # 원하는 폴더의 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result
               
# 병합을 위해 날짜만을 가지고 있는 데이터 프레임 생성
def get_empty_day_df(sqlalchemy_conn):
     
    sql = "SELECT * FROM investing_data.aedkrw내역 where 날짜 > 20211130 and 날짜 < {0}".format(today)
            
    table_data = sqlalchemy_conn.execute(sql) 
    empty_day_df = pd.DataFrame(table_data.fetchall())  # DB내 테이블을 DF로 변환
    
    empty_day_df = empty_day_df.drop(columns=['AEDKRW내역_종가','AEDKRW내역_오픈','AEDKRW내역_고가'
                                      ,'AEDKRW내역_저가','AEDKRW내역_거래량','AEDKRW내역_변동']) # 날짜를 제외한 컬럼 drop
    
    empty_day_df.to_pickle('../pickle/pickle_df/empty/{0}_1년.pkl'.format(saveday))
    
    return empty_day_df


# investing Data로 이루어진 데이터 프레임 추출
def get_sqlalchemy_investing_df(conn, empty_day_df, investing_table_list):
    
    investing_df = pd.DataFrame(empty_day_df)
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
    for table in investing_table_list:
        
        sql = "SELECT * FROM investing_data.`{0}` where 날짜 > 20211130 and 날짜 < {1}".format(table, today)
        table_data= conn.execute(sql)
        table_df = pd.DataFrame(table_data.fetchall())  # DB내 테이블을 DF로 변환
        table_df['날짜'] = table_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
        
        investing_df = pd.merge(investing_df, table_df,on='날짜', how='left') # investing data들을 하나의 df로 제작
        
    # 빈 값 채워주기    
    for c in list(investing_df.columns):
        if c.split('_')[-1] == '거래량' or c.split('_')[-1] == '변동':
            investing_df[c] = investing_df[c].fillna(0)
        else:
            investing_df[c] = investing_df[c].fillna(method='bfill')
            investing_df[c] = investing_df[c].fillna(method='ffill')
            
    return investing_df  

# stock df와 investing df를 병합
def get_sqlalchemy_stock_investing_merge_df(conn, stock_table_list, investing_df):
    
    complete_df = pd.DataFrame()
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
    for table in tqdm(stock_table_list):
        sql = "SELECT * FROM stock_info.`{0}` where 날짜 > 20211130 and 날짜 < {1}".format(table, today)
        table_data = conn.execute(sql)
        stock_df = pd.DataFrame(table_data.fetchall()) # DB내 테이블을 DF로 변환
        
        drop_list = ['외국인주문한도수량','외국인주문가능수량','수정주가일자','수정주가비율']
        stock_df.drop(drop_list,axis=1, inplace=True)
        
        stock_df['날짜'] = stock_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
        merge_df = pd.merge(stock_df, investing_df, on='날짜') # stock df 와 investing df 를 날짜 기준으로 merge

        complete_df = pd.concat([complete_df, merge_df], axis=0) # merge가 된 df들을 concat하여 하나의 df로 제작
        
    conn.close()
    return complete_df      


In [37]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
investing_table_list = get_pymysql_stock_list(pymysql_conn, 'investing_data')
empty_day_df = get_empty_day_df(sqlalchemy_conn)
investing_df = get_sqlalchemy_investing_df(sqlalchemy_conn, empty_day_df, investing_table_list)

In [38]:
stock_table_list = ['005930','373220','207940','000660','051910','247540','091990','066970','293490','028300'] # 대형주
complete_df = get_sqlalchemy_stock_investing_merge_df(sqlalchemy_conn, stock_table_list, investing_df)

100%|██████████| 10/10 [01:09<00:00,  6.98s/it]


In [39]:
day_col_list = ['전일대비','상장주식수','시가총액','외국인현보유수량'
                ,'외국인현보유비율','기관순매수량','기관누적순매수량'
                , '년', '월', '일']   
investing_col_list= list(investing_df.columns.drop('날짜'))

shift_list = day_col_list + investing_col_list

for col in shift_list:
    complete_df[col] = complete_df[col].shift(381)
complete_df.dropna(inplace=True)

In [41]:
investing_df.to_pickle('../pickle/pickle_df/investing/{0}_1년_10개.pkl'.format(saveday))
complete_df.to_pickle('../../data/pickle_complete/대형주_{0}_1년_10개.pkl'.format(saveday))

In [4]:
with open('.../../data/pickle_complete/대형주_20221206_6개월_10개.pkl', 'rb') as f:
    stock_df = pickle.load(f)
# 상관 계수 높은 거 추출
corr_matrix = stock_df.corr()
cor = corr_matrix["pct_label"]
# cor.to_pickle('../pickle/pickle_corr/대형주_{0}_6개월_10개.pkl'.format(saveday))

corr_matrix.to_pickle('../pickle/pickle_corr_matrix/대형주_20221206_6개월_10개.pkl')

In [ ]:
with open('../pickle/pickle_corr/대형주_20221206_1년_10개.pkl', 'rb') as f:
    cor = pickle.load(f)
    
co= co.drop('날짜')
cor1 = co[(co.values>0.04) | (co.values<-0.04) |
           (co.index == '고가') | (co.index == '시가') | 
           (co.index == '종가') | (co.index == '저가')  ]
cor1 = cor1.drop('pct_label')
cor1.to_pickle('../pickle/pickle_corr/대형주_20221206_1년_10개_035.pkl')
len(cor1)

In [10]:
with open('../../data/pickle_complete/대형주_20221206_6개월_10개.pkl', 'rb') as f:
    stock_df = pickle.load(f)
with open('../pickle/pickle_corr_matrix/대형주_20221206_6개월_10개.pkl', 'rb') as f:
    cor_df = pickle.load(f)
    
drop_list = []
for i, cor in tqdm(enumerate(cor_df.columns)):
    for v, j in zip(cor_df.loc[cor].values, cor_df.iloc[i].index ):
        # print(v)
        if (v > 0.8) & (v!=1) & (j not in drop_list):
            drop_list.append(j)
            
cor_list = list(cor_df.columns)
for i in tqdm(drop_list):
    cor_list.remove(i)
    
essential_list = ['날짜','시간','시가','고가', '저가', '종가'] 
for e in tqdm(essential_list):
    if e not in cor_list: 
        cor_list.append(e)


corr_df = stock_df[cor_list].corr()
new_corr = corr_df["pct_label"]
new_corr.to_pickle('../pickle/pickle_corr_complete/대형주_20221206_6개월_10개.pkl')

1287it [00:11, 113.23it/s]
100%|██████████| 6/6 [00:00<?, ?it/s]


In [14]:
with open('../pickle/pickle_corr_complete/대형주_20221206_6개월_10개.pkl', 'rb') as f:
    new_corr = pickle.load(f)

new_corr= new_corr.drop('날짜')
complete_cor = new_corr[(new_corr.values>0.05) | (new_corr.values<-0.05) |
                (new_corr.index == '고가') | (new_corr.index == '시가') | 
                (new_corr.index == '종가') | (new_corr.index == '저가')  ]
complete_cor = complete_cor.drop('pct_label')
complete_cor.to_pickle('../pickle/pickle_corr_complete/대형주_20221206_6개월_10개_05.pkl')
len(complete_cor)

23